# Creating your own dataset from Google Images

*by: Francisco Ingham and Jeremy Howard. Inspired by [Adrian Rosebrock](https://www.pyimagesearch.com/2017/12/04/how-to-create-a-deep-learning-dataset-using-google-images/)*

In this tutorial we will see how to easily create an image dataset through Google Images. **Note**: You will have to repeat these steps for any new category you want to Google (e.g once for dogs and once for cats).

# Downloading Data

Teddy Bear vs Grizzly vs Black

Awesome library to get data from google image search [google-images-download](https://github.com/hardikvasa/google-images-download)

In [ ]:
!pip install google_images_download

In [ ]:
from google_images_download import google_images_download   #importing the library

In [ ]:
response = google_images_download.googleimagesdownload()   #class instantiation

In [1]:
images_to_download = ["grizzly bear", "teddy bear", "black bear"]

#images_to_download = ["pizza", "pasta", "hot dog"]


In [29]:
images_to_download = sorted(images_to_download)
print (images_to_download)
num_images_to_download = 100
output_directory = "data"
safe_search = True

['black bear', 'grizzly bear', 'teddy bear']


In [ ]:
for image_class in images_to_download:
    print (f"Downloading {image_class}")
    # arguments the library will pass to google search
    arguments = {"keywords":image_class,"limit":num_images_to_download
                 ,"print_urls":False, "output_directory": output_directory
                 , "safe_search":safe_search
                , "format": "jpg"
                }   
    
    # passing the arguments to the function
    paths = response.download(arguments)   
    # printing absolute paths of the downloaded images
    #print(paths)   

In [3]:
from fastai import *
from fastai.vision import *

Choose an appropriate name for your labeled images. You can run these steps multiple times to grab different labels.

You will need to run this line once per each category.

In [4]:
path = Path('data')

Then we can remove any images that can't be opened:

In [5]:
image_path = path/"train"

In [6]:
image_path.ls()

[PosixPath('data/train/black bear'),
 PosixPath('data/train/teddy bear'),
 PosixPath('data/train/grizzly bear')]

In [ ]:
for c in images_to_download:
    print(c)
    verify_images(image_path/c, delete=True, max_workers=8)

## View data

In [ ]:
np.random.seed(42)
data = ImageDataBunch.from_folder(path, train="train", valid_pct=0.2,
        ds_tfms=get_transforms(), size=224, num_workers=4).normalize(imagenet_stats)

Good! Let's take a look at some of our pictures then.

In [ ]:
data.classes

In [ ]:
data.show_batch(rows=3, figsize=(7,8))

In [ ]:
data.classes, data.c, len(data.train_ds), len(data.valid_ds)

## Train model

In [ ]:
learn = create_cnn(data, models.resnet34, metrics=error_rate)

In [ ]:
learn.fit_one_cycle(4)

In [ ]:
learn.unfreeze()

In [ ]:
learn.fit_one_cycle(2, max_lr=slice(3e-5,3e-4))

In [12]:
final_model_name = 'final'

In [ ]:
learn.save(final_model_name)

## Interpretation

In [ ]:
learn.load(final_model_name)

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
interp.plot_confusion_matrix()

## Putting your model in production

In [ ]:
data.classes

You probably want to use CPU for inference, except at massive scale (and you almost certainly don't need to train in real-time). If you don't have a GPU that happens automatically. You can test your model on CPU like so:

In [ ]:
# fastai.defaults.device = torch.device('cpu')

In [ ]:
image_path.ls()[0]

In [ ]:
sample_img = image_path.ls()[0].ls()[5]
sample_img

In [ ]:
img = open_image(sample_img)
img

In [13]:
classes = images_to_download


In [ ]:
data2 = ImageDataBunch.single_from_classes(path, classes
                                           , tfms=get_transforms()
                                           , size=224).normalize(imagenet_stats)
learn = create_cnn(data2, models.resnet34)
learn.load(final_model_name)

In [ ]:
data2.classes, data2.c

In [ ]:
pred_class,pred_idx,losses = learn.predict(img)
pred_class

In [ ]:
outputs

In [ ]:
softmax= nn.Softmax()
probabilities = softmax(outputs)
probabilities

In [28]:
ClassificationLearner

fastai.vision.learner.ClassificationLearner

In [27]:
type(model)

fastai.vision.learner.ClassificationLearner

In [23]:
??open_image

Signature: open_image(fn:Union[pathlib.Path, str]) -> fastai.vision.image.Image
Source:   
def open_image(fn:PathOrStr)->Image:
    "Return `Image` object created from image in file `fn`."
    x = PIL.Image.open(fn).convert('RGB')
    return Image(pil2tensor(x).float().div_(255))
File:      /opt/anaconda3/lib/python3.6/site-packages/fastai/vision/image.py
Type:      function


In [14]:
def load_model(path, classes ,  model_name, architecture= models.resnet34, image_size=224):
    data = ImageDataBunch.single_from_classes(path, classes
                                           , tfms=get_transforms()
                                           , size=image_size).normalize(imagenet_stats)
    learn = create_cnn(data, architecture)
    learn.load(model_name)
    return learn

In [15]:
model = load_model(path="data" ,classes=classes, model_name=final_model_name )

In [20]:
doc(ImageDataBunch.single_from_classes)

single_from_classes [source] single_from_classes ( path : PathOrStr , classes : StrList , kwargs ) 
 
 Show in docs

In [16]:
from io import BytesIO

In [17]:
url = "https://upload.wikimedia.org/wikipedia/commons/0/08/01_Schwarzb%C3%A4r.jpg"
response = requests.get(url)
img = open_image(BytesIO(response.content))

In [18]:
type(img)

fastai.vision.image.Image

In [19]:
pred_class,pred_idx,losses = model.predict(img)
pred_class

'black bear'

In [ ]:
losses

In [ ]:
softmax= nn.Softmax()
probabilities

In [ ]:
learn.data.classes

In [ ]:
losses.tolist()

In [ ]:
losses.flatten()

In [ ]:
list(map (float, losses ))

# Predict from url


In [ ]:
doc(open_image)

So you might create a route something like this ([thanks](https://github.com/simonw/cougar-or-not) to Simon Willison for the structure of this code):

```python
@app.route("/classify-url", methods=["GET"])
async def classify_url(request):
    bytes = await get_bytes(request.query_params["url"])
    img = open_image(BytesIO(bytes))
    _,_,losses = learner.predict(img)
    return JSONResponse({
        "predictions": sorted(
            zip(learner.data.classes, map(float, losses)),
            key=lambda p: p[1],
            reverse=True
        )
    })
```

(This example is for the [Starlette](https://www.starlette.io/) web app toolkit.)